In [ ]:
import os, shutil
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing import image

from tensorflow.keras.utils import plot_model

In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'C:/0_thesis/0_dataset-analysis')

from age_groups import *

In [ ]:
batch_size = 64 # !!

img_size = 256
x_col = 'filename'
y_col = ['age', 'gender']

In [ ]:
ds_path = 'C:/0_thesis/dataset/utkface-pepper/'
csv_path = 'C:/0_thesis/dataset/utkface-pepper.csv'
results_folder = "C:/0_thesis/2_model/TESTING/both/pepper-alpha1/19/"
colormode = "rgb" # "rgb" or "grayscale"

pepper_val = "C:/0_thesis/dataset/pepper-validation-data"
pepper_val_csv = "C:/0_thesis/dataset/pepper-validation-data.csv"

pepper_val_gray = "C:/0_thesis/dataset/pepper-validation-data-gray"
pepper_val_csv_gray = "C:/0_thesis/dataset/pepper-validation-data-gray.csv"

is_pepper_test = False

In [ ]:
import random

df = pd.read_csv(csv_path)
n_tot_images = df.shape[0]

gender_mapper = {'male': 0, 'female': 1}
df = df.replace({"gender": gender_mapper})
#df.info()

# Balance dataset
def get_n_rows(n, df):
    rows = random.sample(np.arange(0,len(df.index)).tolist(),n)
    return df.iloc[rows,]

df_new = df[df['age-group'] == 0]

for i in range(1, 12):
    temp = df[df['age-group'] == i]
    if len(temp) <= 2000:
        df_new = pd.concat([df_new, temp], axis=0)
    else:
        df_new = pd.concat([df_new, get_n_rows(2000, temp)], axis=0)

counts = np.bincount(df_new["age-group"])
print(counts)


## EVALUATE ON same validation set used in training

In [ ]:
is_pepper_test = False

validation_data = pd.read_csv(results_folder+"/test_data.csv")
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(validation_data, 
                                                directory = ds_path, 
                                                x_col = x_col, 
                                                y_col = y_col, 
                                                target_size = (img_size, img_size),
                                                class_mode="multi_output",
                                                shuffle = False,
                                                batch_size = batch_size,
                                                color_mode = colormode)

## ON PEPPER PHOTOS

In [ ]:
# VAL ON PEPPER PHOTOS 
if colormode == "rgb":
    csv = pepper_val_csv
    directory = pepper_val
elif colormode == "grayscale":
    csv = pepper_val_csv_gray
    directory = pepper_val_gray
    
is_pepper_test = True

validation_data = pd.read_csv(csv)

gender_mapper = {'male': 0, 'female': 1}
validation_data = validation_data.replace({"gender": gender_mapper})

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(validation_data, 
                                                directory = directory, 
                                                x_col = x_col, 
                                                y_col = y_col, 
                                                target_size = (img_size, img_size),
                                                class_mode="multi_output",
                                                shuffle=False,
                                                batch_size = batch_size,
                                                color_mode=colormode)

In [ ]:
np.bincount(validation_data["gender"])

## Load and evaluate the model

In [ ]:
model = keras.models.load_model(results_folder+"/model_4")

In [ ]:
# Evaluate
model.evaluate(val_generator)

In [ ]:
# Predict
prediction = model.predict(val_generator)
y_pred_age = np.round(prediction[0])
y_pred_gender = np.round(prediction[1])

y_pred_gender = y_pred_gender.astype('int')
y_pred_age = y_pred_age.astype('int')
validation_data.astype({'gender': 'int', 'age': 'int'})

In [ ]:
y_pred_age.tolist()

In [ ]:
y_pred_groups = []

for pred in y_pred_age:
    y_pred_groups.append(AgeGroups().getGroupFromAge(pred))

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [ ]:
# GENDER
precision = precision_score(validation_data["gender"], y_pred_gender)
recall = recall_score(validation_data["gender"], y_pred_gender)
accuracy = accuracy_score(validation_data["gender"], y_pred_gender)
f1 = f1_score(validation_data["gender"], y_pred_gender)
print("Accuracy on gender: ", accuracy)
print("Precision on gender: ", precision)
print("Recall on gender: ", recall)
print("F1 on gender: ", f1)

In [ ]:
# AGE-GROUP
precision = precision_score(validation_data["age-group"], y_pred_groups, average='micro')
recall = recall_score(validation_data["age-group"], y_pred_groups, average='micro')
accuracy = accuracy_score(validation_data["age-group"], y_pred_groups)
f1 = f1_score(validation_data["age-group"], y_pred_groups, average='micro')
print("Accuracy on age-group: ", accuracy)
print("Precision on age-group: ", precision)
print("Recall on age-group: ", recall)
print("F1 on age-group: ", f1)

In [ ]:
# Confusion matrix GENDER
cm = confusion_matrix(validation_data['gender'], y_pred_gender, labels=np.unique(y_pred_gender))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_pred_gender))
disp.plot()
if not is_pepper_test:
    plt.savefig(results_folder+"/cm_gender.jpg")

In [ ]:
cm = confusion_matrix(validation_data["age-group"], y_pred_groups)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(validation_data["age-group"]))
disp.plot()
if not is_pepper_test:
    plt.savefig(results_folder+"/cm_age-groups.jpg")

In [ ]:
# Print 9 random images with PREDICTION
import random

plt.figure(figsize=(10,10))

indices = random.sample(np.arange(0,len(validation_data.index)).tolist(),9)

for j, i in enumerate(indices):
    sample = validation_data.iloc[i]
    
    actual_gender = "Female" if sample.gender==1 else "Male"
    pred_gender = "Female" if y_pred_gender[i]==1 else "Male"
    actual_age = sample['age']
    pred_age = y_pred_age[i]
    
    plt.subplot(3,3,j+1)
    plt.axis('off')
    plt.title('Actual: %s, %s\nPred: %s, %s' % (actual_gender, actual_age, pred_gender, pred_age))
    
    if is_pepper_test:
        if colormode == "rgb":
            plt.imshow(Image.open(pepper_val+"/"+sample.filename))
        elif colormode == "grayscale":
            plt.imshow(Image.open(pepper_val_gray+"/"+sample.filename), cmap='gray')
        plt.savefig(results_folder+"/example_pepper.jpg")
    else:
        if colormode == "rgb":
            plt.imshow(Image.open(ds_path+"/"+sample.filename))
        else:
            plt.imshow(Image.open(ds_path+"/"+sample.filename), cmap='gray')
        plt.savefig(results_folder+"/example.jpg")
plt.show()

## ON EXTERNAL IMAGES

In [ ]:
#results_folder = "C:/0_thesis/2_model/TESTING/both/pepper-alpha1/19fold/model_4"
results_folder = "C:/0_thesis/2_model/TESTING/both/pepper-alpha05/20pepper05/model_4"
#results_folder = "C:/0_thesis/2_model/TESTING/both/no-pepper/20netto/model_4"

inputdir = "C:/0_thesis/3_experiment/results/6/photos/"

!python predict.py --modelpath=$results_folder --imagespath=$inputdir \
--resultfile="C:/0_thesis/2_model/TESTING/prediction.txt" \
--colormode="rgb"

In [ ]:
#results_folder = "C:/0_thesis/2_model/TESTING/both/pepper-alpha1/19foldgray/model_4"
#results_folder = "C:/0_thesis/2_model/TESTING/both/pepper-alpha05/20pepper05gray/model_4"
results_folder = "C:/0_thesis/2_model/TESTING/both/no-pepper/20nettogray/model_4"

inputdir = "C:/0_thesis/3_experiment/results/7/photos/"

!python predict.py --modelpath=$results_folder --imagespath=$inputdir \
--resultfile="C:/0_thesis/2_model/TESTING/prediction.txt" \
--colormode="grayscale"